![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)

<a href="https://hub.callysto.ca/jupyter/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fcallysto%2Flesson-plans&branch=master&subPath=notebooks/data-science-with-covid/data-science-with-covid-student.ipynb&depth=1"><img src="https://raw.githubusercontent.com/callysto/curriculum-notebooks/master/open-in-callysto-button.svg?sanitize=true" width="123" height="24" alt="Open in Callysto"/></a>

# Introduction to Data Science with COVID-19 Data

While there are a number of well-designed dashboards and visualization tools for COVID-19 data, such as [Bing](https://bing.com/covid) and [The World Bank](http://datatopics.worldbank.org/universal-health-coverage/coronavirus/), we are going to try building something ourselves in a Jupyter notebook.

This Jupyter notebook uses [COVID-19 statistics from Johns Hopkins University CSSE](https://github.com/CSSEGISandData/COVID-19), you can also see [their dashboard](https://www.arcgis.com/apps/opsdashboard/index.html#/bda7594740fd40299423467b48e9ecf6).

## Licence and Disclaimer

COVID-19 data sets are copyright 2020 [Johns Hopkins University](https://systems.jhu.edu) (available for educational and academic research purposes). The population data is free to use from [Gapminder](https://www.gapminder.org) under a [Creative Commons Attribution license](https://creativecommons.org/licenses/by/4.0/). This notebook also carries a [Creative Commons Attribution license](https://creativecommons.org/licenses/by/4.0/).

This notebook should not be considered medical or policy-making advice. Always follow the directives and orders of your public health authority.

## Getting Started

First, `▶Run` the next cell to import a data set. Once the data set has been downloaded and imported into a [DataFrame](https://www.tutorialspoint.com/python_pandas/python_pandas_dataframe.htm), it will be displayed.

You can change the date, but make sure you use the format `'MM-DD-YYYY'` as they do in the CSSE data set. Files are updated once a day around midnight [UTC](https://en.wikipedia.org/wiki/Coordinated_Universal_Time).

In [ ]:
date = '04-08-2020'

import pandas as pd
import plotly.graph_objects as go

csv_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/'+date+'.csv'
covid_stats = pd.read_csv(csv_url)
covid_stats

## Data Cleaning

`Run` the next cell to clean up the data. We'll add up values for each country and create a new dataframe. If you prefer specific countries, remove the `#` from the second line and edit that list of countries.

In [ ]:
country_list = covid_stats['Country_Region'].unique()
#country_list = ['Italy', 'Spain', 'Germany', 'France', 'Israel', 'US', 'United Kingdom', 'Singapore', 'Australia', 'Canada', 'China', 'Argentina', 'Russia', 'India']
df_list = []

df = pd.DataFrame()
df['Country'] = country_list
df['Confirmed'] = df['Country'].apply(lambda x: covid_stats[covid_stats['Country_Region']==x]['Confirmed'].sum())
df['Recovered'] = df['Country'].apply(lambda x: covid_stats[covid_stats['Country_Region']==x]['Recovered'].sum())
df['Deaths'] = df['Country'].apply(lambda x: covid_stats[covid_stats['Country_Region']==x]['Deaths'].sum())
df = df.sort_values('Confirmed',ascending=False)
df

## Graphing Data

We will use the [Plotly Express](https://plotly.com/python/plotly-express/) library to create a graph of our data set.

In [ ]:
import plotly.express as px
px.bar(df.head(20), x='Country', y='Confirmed', title='COVID Cases by Country')

### Renaming and Listing Countries

The country/state naming for this data set mostly follows the [WHO list of member states](https://www.who.int/choice/demography/by_country/en/), but we can run the next cell to rename some of them. Then we will list the unique country names in the data set.

In [ ]:
df.replace('US','United States',regex=True,inplace=True)
df.replace('Korea, South','South Korea',regex=True,inplace=True)

df['Country'].unique()

## Adding World Data

We can also add up all of the values in the data set to get worldwide totals.

In [ ]:
confirmed = covid_stats['Confirmed'].sum()
recovered = covid_stats['Recovered'].sum()
deaths = covid_stats['Deaths'].sum()
world_values = {'Country': 'World', 'Confirmed': confirmed, 'Recovered': recovered, 'Deaths': deaths}
world_df = pd.DataFrame([world_values])
df = pd.concat([df, world_df], ignore_index=True)
df.tail()

## Sorting Data

`Run` the next cell to sort the data by a particular column using the code:

```python
df.sort_values('Confirmed', ascending=False).head(16)
```

The `ascending=False` is optional (the default is `True`), and `.head(16)` shows just the first 16 rows.

## Selecting Specific Countries

To see a DataFrame of specific countries, use either of the following two options:

```python
df[df['Country']=='Canada']
```

or

```python
list_of_countries = ['Canada', 'China', 'Italy']
df[df['Country'].isin(list_of_countries)]
```

## Adding Population Data

We'll use population data from [Gapminder](https://gapminder.org).

In [ ]:
pop_sheet_id = '18Ep3s1S0cvlT1ovQG9KdipLEoQ1Ktz5LtTTQpDcWbX0'
pop_gid = '1668956939'
population_csv_url = 'https://docs.google.com/spreadsheets/d/'+pop_sheet_id+'/export?gid='+pop_gid+'&format=csv'
population_data = pd.read_csv(population_csv_url)
population = population_data[population_data['time']==2019]
population

Set the index as country name for both dataframes so we can join them together.

In [ ]:
cp = population.set_index('name')
cs = df.set_index('Country')
new_df = cs.join(cp)
new_df

Drop some columns we don't need, and rename for consistency.

In [ ]:
new_df.drop(columns=['geo','time'],inplace=True)
new_df.rename(columns={'population':'Population'},inplace=True)
new_df

Drop any "not available" data.

In [ ]:
new_df = new_df.dropna()
new_df

Calculate values for a new column called `Confirmed Percent`.

In [ ]:
pd.options.mode.chained_assignment = None
new_df.loc[:, 'Confirmed Percent'] = new_df['Confirmed'] / new_df['Population'] * 100
new_df

Make a graph. The index (Country) will be on the x-axis by default.

In [ ]:
y_values = 'Confirmed Percent'
px.bar(new_df.sort_values(y_values,ascending=False).head(20), y=y_values, title=y_values+' of Population')

## Next Steps

Hopefully that's an interesting introduction to data science using online COVID-19 data.

If you would like to see time series or geographical data, here are some examples.

In [ ]:
time_series_confirmed_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
tsc = pd.read_csv(time_series_confirmed_url)

time_series_deaths_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
tsd = pd.read_csv(time_series_deaths_url)

time_series_recovered_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'
tsr = pd.read_csv(time_series_recovered_url)

tsr

Below is an example of a time-series visualization using COVID-19 data.

In [ ]:
country_traces = []
for country in tsr['Country/Region'].unique():
    country_data = tsr[tsr['Country/Region'] == country].set_index('Country/Region').drop(columns=['Province/State', 'Lat', 'Long']).T
    trace = go.Scatter(x=country_data.index, y=country_data[country], mode='lines', name=country)
    country_traces.append(trace)

layout = go.Layout(title='Covid-19 Results per Country', xaxis=dict(title='Date'), yaxis=dict(title='Confirmed'))
covid_fig = go.Figure(data=country_traces, layout=layout).show()

Plot geospacial data

In [ ]:
px.scatter_geo(covid_stats, lat='Lat', lon='Long_', size='Confirmed', hover_name='Combined_Key')

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)